Calculate mean and std of images dataset.

Source: https://discuss.pytorch.org/t/about-normalization-using-pre-trained-vgg16-networks/23560/6

In [1]:
import pandas as pd
from loader import ImageDS_Stats
import torch
import torch.utils.data as D
import matplotlib.pyplot as plt

# Dataset mean and std

In [2]:
df = pd.read_csv('/storage/aptosplus/trainLabels_aug.csv')
df.head()

,id_code,diagnosis,ds
0,000c1434d8d7,2,aptos
1,001639a390f0,4,aptos
2,0024cdab0c1e,1,aptos
3,002c21358ce6,0,aptos
4,005b95c28852,0,aptos


In [3]:
df.tail()

,id_code,diagnosis,ds
143802,44349_right_a5,1,eyepacs
143803,44349_right_a6,1,eyepacs
143804,44349_right_a7,1,eyepacs
143805,44349_right_a8,1,eyepacs
143806,44349_right_a9,1,eyepacs


In [4]:
dataset = ImageDS_Stats(df)

In [5]:
loader = D.DataLoader(
    dataset,
    batch_size=10,
    num_workers=1,
    shuffle=False
)

mean = 0.
std = 0.
nb_samples = 0.

for data, _ in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

In [6]:
print(mean)

tensor([-2.1120, -2.0382, -1.8050])


In [7]:
print(std)

tensor([0.0031, 0.0035, 0.0036])
